In [1]:
from ift6758.pipeline import ExperimentPipeline, DEFAULT_TRANSFORMATIONS
from importlib import reload 
import ift6758.pipeline

In [2]:
reload(ift6758.pipeline.features)

from ift6758.pipeline.features import (
    mirror_coordinates,
    append_shot_angle,
    append_shot_distance,
    replace_nan_by_0,
    append_game_secs,
    append_time_lapse_prev,
    append_dist_prev,
    append_rebound,
    append_angle_change,
    append_speed
)

In [4]:
%%time
trans = [
    mirror_coordinates,
    append_shot_angle,
    append_shot_distance,
    replace_nan_by_0,
    append_game_secs,
    append_time_lapse_prev,
    append_dist_prev,
    append_rebound,
    append_angle_change,
    append_speed
]
df = ExperimentPipeline.get_data('data/tabular', transformations=trans)
df.head()

applying mirror_coordinates
applying append_shot_angle
applying append_shot_distance
applying replace_nan_by_0
applying append_game_secs
applying append_time_lapse_prev
applying append_dist_prev
applying append_rebound
applying append_angle_change
applying append_speed
CPU times: user 7.5 s, sys: 1.05 s, total: 8.55 s
Wall time: 8.58 s


,Unnamed: 0,game_starttime,game_endtime,game_id,datetime,offense_team_id,offense_team_name,offense_team_tricode,goal,x_coords,...,strength_powerplay,season,shot_angle,shot_distance,game_secs,time_lapsed_prev_event_in_seconds,dist_prev_event,rebound,angle_change,speed
0,0,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,2016-10-12T23:19:59Z,10,Toronto Maple Leafs,TOR,0,77.0,...,0.0,20162017,67.380135,13.000000,71,0,138.924440,False,None,inf
1,1,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,2016-10-12T23:21:41Z,9,Ottawa Senators,OTT,0,86.0,...,0.0,20162017,12.994617,13.341664,173,0,36.715120,False,None,inf
2,2,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,2016-10-12T23:23:17Z,9,Ottawa Senators,OTT,0,23.0,...,0.0,20162017,60.068488,76.157731,241,18,102.318131,False,None,5.684341
3,3,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,2016-10-12T23:24:02Z,9,Ottawa Senators,OTT,0,33.0,...,0.0,20162017,75.004921,57.974132,286,19,45.880279,False,None,2.414752
4,4,2016-10-12T23:00:00Z,2016-10-13T01:56:40Z,0,2016-10-12T23:27:30Z,10,Toronto Maple Leafs,TOR,0,34.0,...,0.0,20162017,63.019769,61.717096,406,15,63.348244,False,None,4.223216


In [3]:
import os
from getpass import getpass

os.environ['COMET_API_KEY'] = getpass()

In [4]:
from sklearn.linear_model import LogisticRegression

pipeline = ExperimentPipeline(
    tabular_dir='../data/tabular',
    feature_columns=['shot_distance', 'shot_angle'],
    target_column='goal',
    pipeline_steps=[
        ('logreg', LogisticRegression()),
    ],
    dataset_transformations=DEFAULT_TRANSFORMATIONS,
    parameter_grid=[
        {'logreg__penalty': ['l2'], 'logreg__C': [0.1, 1, 10]},
    ],
    enable_comet=False
)

In [5]:
pipeline.run()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 2/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 3/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 4/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 5/5] END .logreg__C=0.1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 1/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 2/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 3/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 4/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 5/5] END ...logreg__C=1, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 1/5] END ..logreg__C=10, logreg__penalty=l2;, score=0.905 total time=   0.9s
[CV 2/5] END ..logreg__C=10, logreg__penalty=l2;,

In [6]:
pipeline.grid.best_score_

0.9050868966599304

In [7]:
str(pipeline.grid.best_estimator_)

"Pipeline(steps=[('logreg', LogisticRegression(C=0.1))])"